<a href="https://colab.research.google.com/github/datduyng/cs411-data-modeling/blob/master/hw03/Assignment03Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from time import time
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
from google.colab import files
import io
from tqdm import tqdm as loader
import sys
import time
import copy


%matplotlib inline
plt.rcParams['figure.figsize'] = (20,15) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')


# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movies_metadata.csv to movies_metadata (2).csv


In [ ]:
movies= pd.read_csv(io.BytesIO(uploaded['movies_metadata.csv']))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:

#retain numeric columns for clustering
movies = movies[['budget','popularity','revenue','runtime','vote_average','vote_count']]

# drop rows with null or NaN values
movies = movies[~movies.isnull().any(axis=1)]
movies = movies.dropna()

# remove movies with less than 30 votes
movies = movies[movies['vote_count'] > 30]
movies.head()

,budget,popularity,revenue,runtime,vote_average,vote_count
0,30000000,21.9469,373554033.0,81.0,7.7,5415.0
1,65000000,17.0155,262797249.0,104.0,6.9,2413.0
2,0,11.7129,0.0,101.0,6.5,92.0
3,16000000,3.85949,81452156.0,127.0,6.1,34.0
4,0,8.38752,76578911.0,106.0,5.7,173.0


In [ ]:
scaler = StandardScaler()
movies_standardized = scaler.fit_transform(movies.values)

In [ ]:
movies.columns

Index(['budget', 'popularity', 'revenue', 'runtime', 'vote_average',
       'vote_count'],
      dtype='object')

In [ ]:
# # Extra Credit 

# def euclidean(list_A, list_B):
#   diff_vect = list_A - list_B
#   return np.sqrt(np.dot(diff_vect,diff_vect))


# def manhattan(list_A, list_B):
#   # list_length_check(list_A,list_B)
#   from numpy import sqrt
#   a_b = zip(list_A, list_B)
#   manhattan = 0
#   for x,y in a_b:
#       manhattan += abs(x-y)
#   return manhattan

# #store all distances
# def Distance_matrix(X, distance_metric):
#   manhat = np.zeros(shape=(len(X),len(X)))
#   euclid = np.zeros(shape=(len(X),len(X)))
#   for i in range(len(X)):
#     for j in range(len(X)):
#       if euclid[i][j] == 0 & manhat[i][j] == 0 & i!=j:
#         if distance_metric is 'euclidean':
#           euclid[i][j] = euclidean(X[i],X[j])
#           euclid[j][i] = euclid[i][j]
#         if distance_metric is 'manhattan':
#           manhat[i][j] = manhattan(X[i],X[j])
#           manhat[j][i] = manhat[i][j]
#   return euclid, manhat

#   def RangeQueryX(X, distance_metric, euclid, manhat, point, eps ):
#     N=[]
#     idx=[]
#     dist = 100
#     if distance_metric is 'euclidean':
#       idx = (euclid[point]<= eps).nonzero()[0]
#     if distance_metric is 'manhattan':
#       idx=(manhat[point]<= eps).nonzero()[0]
#     N.append(idx)
#     return N

In [ ]:
def euclidean(list_A, list_B):
  diff_vect = list_A - list_B
  return np.sqrt(np.dot(diff_vect,diff_vect))

def manhattan(list_A, list_B):
  from numpy import sqrt
  a_b = zip(list_A, list_B)
  manhattan = 0
  for x,y in a_b:
      manhattan += abs(x-y)
  return manhattan

#   euclidean = lambda q,p: np.sqrt(np.sum((q - p)**2, axis=1))#Stack. vertically

# manhattan = lambda x,y: np.sum(np.abs(x - y), axis=1)
 

def RangeQuery(X, distance_metric, point, eps):
  N = []
  dist = 100
  for i in (range(len(X))):
      if distance_metric is 'euclidean':
        dist = euclidean(X[i],point)
      if distance_metric is 'manhattan':
        dist = manhattan(X[i],point)
      if dist <= eps:
        N.append(i)
  return N

In [ ]:
test = euclidean(movies_standardized[0],movies_standardized[1])

test

3.881588941533921

In [ ]:
#sanity check
test = RangeQuery(movies_standardized, 'euclidean', movies_standardized[2], 10)
len(test)

12067

In [ ]:
class DBSCAN():
    def fit(self, X, eps, min_samples, distance_metric):
        C=0
        label = dict.fromkeys(range(len(X)), "unvisited")
        for i in range(len(X)):
          # sys.stdout.write( '\r'+"currently in "+str(i)+"/"+str(len(X)) +" i.e. "+str((i*100)/len(X)))
          if (label[i] != "unvisited"): 
            continue
          N = RangeQuery(X, distance_metric, X[i], eps)
          if (len(N)<min_samples):
            label[i] = -1
            continue
          C = C+1
          label[i]= C
          N.remove(i)
          S = copy.deepcopy(N)
          for Q in range(len(S)):
              sys.stdout.write( '\r'+"currently in inner loop for iteration "+str(i)+ " and "+str(Q)+"/"+str(len(S)) +" i.e. "+str((Q*100)/len(S)))
              if label[S[Q]] == -1:
                label[S[Q]] = C
              if label[S[Q]] != "unvisited":
                continue
              label[S[Q]] = C
              N = RangeQuery(X, distance_metric, X[Q], eps)
              if len(N) >= min_samples:
                  S.append(N)
        return list(label.values())

    def __init__(self):
            pass
            

In [ ]:
#Sanity Check
dbscan = DBSCAN()
labels = dbscan.fit(movies_standardized[:200], 10,4,'euclidean')
labl = list(set(labels.values()))
len(labl)

currently in inner loop for iteration 0 and 197/394 i.e. 50.0

AttributeError: ignored

In [ ]:
labels_set = set(labels.values())
# len(labels_set)
labels_set

{1}

### Find Optimal EPS  
calculate all distance and sort in ascending order.  
plot the distances on a graph.  
find the elbow or transition point to be the optimal eps

In [ ]:
#findOptimal EPS
dist = np.zeros(shape=(len(movies_standardized),len(movies_standardized)))
for i in loader(range(len(movies_standardized))):
  sys.stdout.write( '\r'+"currently in "+str(i))

  for j in range(len(movies_standardized)):
    dist[i][j] = euclidean(movies_standardized[i],movies_standardized[j])

# distdf=pd.DataFrame(np.sort(dist.values, axis=0), index=dist.index, columns=dist.columns)


  1%|          | 80/12178 [00:05<14:58, 13.47it/s]


KeyboardInterrupt: ignored

In [ ]:
dist_sorted = np.sort(dist, axis=None)  

In [ ]:
plt.plot(dist_sorted)
plt.ylabel(' Distances')
plt.show()

### Run DBSCAN 

In [ ]:
eps = [10.0, 20, 30]
min_samples = [4,6,8]
distance_metric = ['euclidean','manhattan']
X = copy.deepcopy(movies_standardized[:2000])

dbscan_basket = []
best_param, best_score, best_labels = None, -1., []

In [ ]:
for e in eps:
    for ms in min_samples:
        print("current iteration "+ str(e) + " and minsamp "+ str(ms)+"       ")
        for dm in distance_metric:
            dbscan = DBSCAN()
            labels = dbscan.fit(X,e,ms,dm)
            num_labels = len(set(labels))
            print("    num label is "+ str(num_labels))
            if num_labels < 2: continue
            sil = silhouette_score(X, labels)
            params = {
                'eps': e,
                'min_samples': ms,
                'distance_metric': dm,
                'num_labels': num_labels,
                'silhouette': sil
            }

            dbscan_basket.append(params)
            if sil > best_score:
                best_param = params
                best_score = sil
                best_labels = labels

current iteration 10.0 and minsamp 4       
currently in inner loop for iteration 187 and 2/3 i.e. 66.66666666666667    num label is 3
currently in inner loop for iteration 515 and 704/706 i.e. 99.71671388101983    num label is 8
current iteration 10.0 and minsamp 6       
currently in inner loop for iteration 0 and 1996/3991 i.e. 50.01252818842396    num label is 2
currently in inner loop for iteration 515 and 704/706 i.e. 99.71671388101983    num label is 8
current iteration 10.0 and minsamp 8       
currently in inner loop for iteration 0 and 1996/3991 i.e. 50.01252818842396    num label is 2
currently in inner loop for iteration 515 and 704/706 i.e. 99.71671388101983    num label is 8
current iteration 20 and minsamp 4       
currently in inner loop for iteration 0 and 1998/3997 i.e. 49.98749061796347    num label is 1
currently in inner loop for iteration 980 and 2/3 i.e. 66.66666666666667    num label is 3
current iteration 20 and minsamp 6       
currently in inner loop for iter

In [ ]:
print(best_param)

{'eps': 10.0, 'min_samples': 6, 'distance_metric': 'euclidean', 'num_labels': 2, 'silhouette': 0.8610160235477692}


In [ ]:
dbscan_basket.sort(key=lambda x: float(x['silhouette']), reverse=True)
table = {
    'eps':[],
    'min_samples':[],
    'distance_metric':[],
    'num_labels':[],
    'silhouette': []
}

for k, v in table.items():
    table[k] = [row[k] for row in dbscan_basket]

df = pd.DataFrame(table)
pd.set_option('display.max_rows', None, 'display.max_columns', None)
df

,eps,min_samples,distance_metric,num_labels,silhouette
0,10.0,6,euclidean,2,0.861016
1,10.0,8,euclidean,2,0.861016
2,10.0,4,euclidean,3,0.852891
3,20.0,4,manhattan,3,0.852891
4,20.0,6,manhattan,3,0.852891
5,20.0,8,manhattan,3,0.852891
6,10.0,4,manhattan,8,0.456555
7,10.0,6,manhattan,8,0.456555
8,10.0,8,manhattan,8,0.456555
